In [ ]:
from typing import List, Tuple

import numpy as np
import openmm
import openmm.app
from openff.interchange import Interchange
from openff.interchange.interop.openmm import to_openmm_positions
from openff.toolkit import ForceField, Molecule, Topology
from openff.units import unit
from openff.units.openmm import ensure_quantity
from openmm import unit as openmm_unit

In [ ]:
def prepare_simulation(
    openmm_topology: openmm.app.Topology,
    openmm_system: openmm.System,
    positions: unit.Quantity,
    trj_freq: int = 100,
    data_freq: int = 100,
) -> openmm.app.Simulation:
    """Propagate an OpenMM System with Langevin dynamics."""
    time_step = 2 * openmm_unit.femtoseconds
    temperature = 300 * openmm_unit.kelvin
    friction = 1 / openmm_unit.picosecond
    integrator = openmm.LangevinIntegrator(temperature, friction, time_step)

    simulation = openmm.app.Simulation(openmm_topology, openmm_system, integrator)

    assert positions.shape[0] == openmm_topology.getNumAtoms()
    # n_virtual_sites = sum(atom.element is None for atom in openmm_topology.atoms())
    # _positions = np.concatenate(
    #    (molecule.conformers[0], np.zeros((n_virtual_sites, 3))), axis=0
    # )
    # positions = to_openmm(_positions)
    simulation.context.setPositions(ensure_quantity(positions, "openmm"))

    # It's important to run energy minimization before computing velocities; otherwise the initial
    # velocities may be too high as a result of high initial forces, causing a crash
    # See https://github.com/openmm/openmm/issues/3736#issuecomment-1217250635
    simulation.minimizeEnergy()

    # Since we placed all virtual sites at [0.0, 0.0, 0.0], compute virtual site positions to avoid a crash
    simulation.context.computeVirtualSites()

    simulation.context.setVelocitiesToTemperature(temperature)

    pdb_reporter = openmm.app.PDBReporter("trajectory.pdb", trj_freq)
    state_data_reporter = openmm.app.StateDataReporter(
        "data.csv",
        data_freq,
        step=True,
        potentialEnergy=True,
        temperature=True,
        density=True,
    )
    simulation.reporters.append(pdb_reporter)
    simulation.reporters.append(state_data_reporter)

    return simulation


def run_simulation(simulation: openmm.app.Simulation, num_steps: int = 1000):
    import time

    print("Starting simulation")
    start = time.process_time()

    simulation.step(num_steps)

    end = time.process_time()
    print("Elapsed time %.2f seconds" % (end - start))
    print("Done!")

## Part 1: Adding virtual sites to a ligand

In [ ]:
vsite_offxml = """<?xml version="1.0" encoding="utf-8"?>
<SMIRNOFF version="0.3" aromaticity_model="OEAroModel_MDL">
    <VirtualSites version="0.3">
        <VirtualSite
            type="DivalentLonePair"
            name="EP"
            smirks="[*:2]-[#16X2:1]-[*:3]"
            distance="0.70 * angstrom"
            charge_increment1="0.1205*elementary_charge"
            charge_increment2="0.0*elementary_charge"
            charge_increment3="0.1205*elementary_charge"
            sigma="0.1*angstrom"
            epsilon="0.0*kilocalories_per_mole"
            outOfPlaneAngle="54.71384225*degree"
            match="all_permutations" >
        </VirtualSite>
        <VirtualSite
            type="BondCharge"
            name="EP"
            smirks="[*:2][Cl:1]"
            distance="0.4*angstrom"
            charge_increment1="0.2*elementary_charge"
            charge_increment2="0.0*elementary_charge"
            sigma="0.1*angstrom"
            epsilon="0.05*kilocalories_per_mole"
            match="all_permutations" >
        </VirtualSite>
        <VirtualSite
            type="BondCharge"
            name="EP"
            smirks="[*:2][F:1]"
            distance="0.4*angstrom"
            charge_increment1="0.2*elementary_charge"
            charge_increment2="0.0*elementary_charge"
            sigma="0.1*angstrom"
            epsilon="0.05*kilocalories_per_mole"
            match="all_permutations" >
        </VirtualSite>
    </VirtualSites>
</SMIRNOFF>
"""
force_field = ForceField("openff-2.0.0.offxml", vsite_offxml)

In [ ]:
molecule = Molecule.from_smiles("c1cc(Cl)ccc1C(=O)CS[C]1=CO[C](F)(F)CC1")
molecule.generate_conformers(n_conformers=1)
molecule.visualize()

In [ ]:
# Create an Interchange object, which stores information needed for OpenMM (and other engines)
# to understand virtual sites as applied by a force field

interchange = Interchange.from_smirnoff(
    force_field=force_field, topology=molecule.to_topology()
)

assert "VirtualSites" in interchange.handlers.keys()

At this point, each of our OpenFF objects have processed and stored all of the information needed to run a simulation in OpenMM. Next we need to do some conversions prepare the OpenMM objects. First we'll make an _OpenMM_ `Topology` and a corresponding _OpenMM_ `System`, each containing virtual sites.

In [ ]:
# Note that interchange.topology.to_openmm() uses a different code path that DOES NOT include virtual sites
openmm_topology: openmm.app.Topology = interchange.to_openmm_topology()

openmm_system: openmm.System = interchange.to_openmm(combine_nonbonded_forces=True)

In [ ]:
# Retrieve the number of virtualsites in this system/topology by counting
# the number of "Atoms" in the OpenMM `Toppology` with a `None` element
# (this method returns virtual sites as welll, despite the name) ...
n_virtual_sites = sum(atom.element is None for atom in openmm_topology.atoms())

# or the number of particles in the OpenMM `System` with zero mass
assert n_virtual_sites == sum(
    openmm_system.getParticleMass(index)._value == 0
    for index in range(openmm_system.getNumParticles())
)

# This can also be done by inspecting the virtual site handler in the Interchange object
assert n_virtual_sites == len(interchange["VirtualSites"].slot_map)

print(f"There are {n_virtual_sites} virtual particles in this topology.")

In [ ]:
simulation = prepare_simulation(
    openmm_topology=openmm_topology,
    openmm_system=openmm_system,
    positions=to_openmm_positions(interchange),
)

run_simulation(simulation, 10000)

In [ ]:
import mdtraj
import nglview

# Visualize the trajectory. There will probably be errant bonds drawn between atoms and virtual sites.
nglview.show_mdtraj(mdtraj.load("trajectory.pdb"))

## Part 2: Solvating this ligand in TIP4P water

In [ ]:
from openff.interchange.tests import get_test_file_path

force_field_tip4p = ForceField(
    "openff_unconstrained-2.0.0.offxml",
    vsite_offxml,
    get_test_file_path("tip4p.offxml"),
)

In [ ]:
ForceField("openff-1.0.0.offxml")["LibraryCharges"].parameters

In [ ]:
force_field_tip4p["LibraryCharges"].parameters

In [ ]:
force_field_tip4p["VirtualSites"].parameters[-1]

In [ ]:
import numpy
from openff.evaluator.utils.packmol import pack_box

# Construct a water molecule
water = Molecule.from_smiles("O")
water.generate_conformers(n_conformers=1)

# Estimate 80% of the density of waters that would solvate this ligand in a 3 nm box
box_size = unit.Quantity(3.0 * numpy.ones(3), unit.nanometer)
box_volume = numpy.prod(box_size)
water_molecule_volume = 18 * unit.amu / (800 * unit.kilogram / unit.meter**3)

num_water = int((box_volume / water_molecule_volume).to(unit.dimensionless))

interchange.to_pdb("ligand.pdb")

packed_trj, _ = pack_box(
    molecules=[water],
    number_of_copies=[num_water],
    structure_to_solvate="ligand.pdb",
    box_size=box_size,
)

xyz = packed_trj.xyz[0] * unit.nanometer
box = packed_trj.unitcell_vectors[0] * unit.nanometer

In [ ]:
topology = Topology.from_molecules([molecule, *num_water * [water]])

topology.box_vectors = box
topology.set_positions(xyz)

In [ ]:
solvated_interchange = Interchange.from_smirnoff(
    force_field=force_field_tip4p,
    topology=topology,
)

In [ ]:
Interchange.from_smirnoff(
    force_field=ForceField(
        get_test_file_path("tip4p.offxml"),
        "openff_unconstrained-2.0.0.offxml",
        vsite_offxml,
    ),
    topology=topology,
)["VirtualSites"].slot_map

In [ ]:
simulation = prepare_simulation(
    openmm_topology=solvated_interchange.to_openmm_topology(),
    openmm_system=solvated_interchange.to_openmm(combine_nonbonded_forces=True),
    positions=to_openmm_positions(solvated_interchange),
    trj_freq=10,
)

run_simulation(simulation, 1000)

In [ ]:
trajectory = mdtraj.load("trajectory.pdb")

view = nglview.show_mdtraj(trajectory)
view.clear_representations()
view.add_representation(
    "ball+stick",
    selection=[*range(trajectory.topology.residue(0).n_atoms)],
)
view.add_representation(
    "spacefill",
    radius=0.4,
    selection=[
        *range(trajectory.topology.residue(0).n_atoms, trajectory.topology.n_atoms)
    ],
)
view